# Keypoint(Patch) Description
  
This project will be all about defining and training a convolutional neural network to perform keypoint  description. 
PyTorch tutorials are available at: https://github.com/yunjey/pytorch-tutorial


The first step is to load and visualize the data you'll be working with. 

We will use below dataset in this project:
###  The Photo Tourism dataset 
(http://phototour.cs.washington.edu/patches/default.htm)

It is also available in PyTorch torchvision datasets:
https://pytorch.org/docs/stable/_modules/torchvision/datasets/phototour.html#PhotoTour

This dataset consists of 1024 x 1024 bitmap (.bmp) images, each containing a 16 x 16 array of image patches. Here are some examples:

<table><tr><td><img src='images/patches0001.bmp'></td><td><img src='images/patches1482.bmp'></td></tr></table>    
For details of how the scale and orientation is established, please see the paper:  
<p class="style8"><font size="2">S. Winder and M. Brown. <strong>Learning Local Image 
				Descriptors</strong>. To appear <i>International Conference on 
				Computer Vision and Pattern Recognition (CVPR2007)</i> (</font><a href="http://research.microsoft.com/~swinder/papers/winder_brown_cvpr07.pdf"><span class="style9">pdf 
				300Kb</span></a><font size="2">)</font></p>



---

### Import packages

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
print("Done!")
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print("Done!")
class TripletPhotoTour(dset.PhotoTour):
    """
    From the PhotoTour Dataset it generates triplet samples
    note: a triplet is composed by a pair of matching images and one of
    different class.
    """
    def __init__(self, train=True, transform=None, batch_size = None,load_random_triplets = False,  *arg, **kw):
        super(TripletPhotoTour, self).__init__(*arg, **kw)
        self.transform = transform
        self.out_triplets = load_random_triplets
        self.train = train
        self.n_triplets = args.n_triplets
        self.batch_size = batch_size

        if self.train:
            print('Generating {} triplets'.format(self.n_triplets))
            self.triplets = self.generate_triplets(self.labels, self.n_triplets)

    @staticmethod
    def generate_triplets(labels, num_triplets):
        def create_indices(_labels):
            inds = dict()
            for idx, ind in enumerate(_labels):
                if ind not in inds:
                    inds[ind] = []
                inds[ind].append(idx)
            return inds

        triplets = []
        indices = create_indices(labels.numpy())
        unique_labels = np.unique(labels.numpy())
        n_classes = unique_labels.shape[0]
        # add only unique indices in batch
        already_idxs = set()

        for x in tqdm(range(num_triplets)):
            if len(already_idxs) >= args.batch_size:
                already_idxs = set()
            c1 = np.random.randint(0, n_classes)
            while c1 in already_idxs:
                c1 = np.random.randint(0, n_classes)
            already_idxs.add(c1)
            c2 = np.random.randint(0, n_classes)
            while c1 == c2:
                c2 = np.random.randint(0, n_classes)
            if len(indices[c1]) == 2:  # hack to speed up process
                n1, n2 = 0, 1
            else:
                n1 = np.random.randint(0, len(indices[c1]))
                n2 = np.random.randint(0, len(indices[c1]))
                while n1 == n2:
                    n2 = np.random.randint(0, len(indices[c1]))
            n3 = np.random.randint(0, len(indices[c2]))
            triplets.append([indices[c1][n1], indices[c1][n2], indices[c2][n3]])
        return torch.LongTensor(np.array(triplets))

    def __getitem__(self, index):
        def transform_img(img):
            if self.transform is not None:
                img = self.transform(img.numpy())
            return img

        if not self.train:
            m = self.matches[index]
            img1 = transform_img(self.data[m[0]])
            img2 = transform_img(self.data[m[1]])
            return img1, img2, m[2]

        t = self.triplets[index]
        a, p, n = self.data[t[0]], self.data[t[1]], self.data[t[2]]

        img_a = transform_img(a)
        img_p = transform_img(p)
        img_n = None
        if self.out_triplets:
            img_n = transform_img(n)
        # transform images if required
        if args.fliprot:
            do_flip = random.random() > 0.5
            do_rot = random.random() > 0.5
            if do_rot:
                img_a = img_a.permute(0,2,1)
                img_p = img_p.permute(0,2,1)
                if self.out_triplets:
                    img_n = img_n.permute(0,2,1)
            if do_flip:
                img_a = torch.from_numpy(deepcopy(img_a.numpy()[:,:,::-1]))
                img_p = torch.from_numpy(deepcopy(img_p.numpy()[:,:,::-1]))
                if self.out_triplets:
                    img_n = torch.from_numpy(deepcopy(img_n.numpy()[:,:,::-1]))
        if self.out_triplets:
            return (img_a, img_p, img_n)
        else:
            return (img_a, img_p)

    def __len__(self):
        if self.train:
            return self.triplets.size(0)
        else:
            return self.matches.size(0)
print("Done!")
def create_loaders(dataset_names, load_random_triplets = False):
    test_dataset_names = copy(dataset_names)
    test_dataset_names.remove(args.training_set)

    kwargs = {'num_workers': args.num_workers, 'pin_memory': args.pin_memory} if args.cuda else {}

    #np_reshape64 = lambda x: np.reshape(x, (16, 16, 1))
    np_reshape64 = lambda x: np.reshape(x, (64, 64, 1))
    transform_test = transforms.Compose([
            transforms.Lambda(np_reshape64),
            transforms.ToPILImage(),
            transforms.Resize(32),
            transforms.ToTensor()])
    transform_train = transforms.Compose([
            transforms.Lambda(np_reshape64),
            transforms.ToPILImage(),
            transforms.RandomRotation(5,PIL.Image.BILINEAR),
            #transforms.RandomResizedCrop(8, scale = (0.9,1.0),ratio = (0.9,1.1)),
            #transforms.Resize(8),
            transforms.RandomResizedCrop(32, scale = (0.9,1.0),ratio = (0.9,1.1)),
            transforms.Resize(32),
            transforms.ToTensor()])
    transform = transforms.Compose([
            transforms.Lambda(cv2_scale),
            transforms.Lambda(np_reshape),
            transforms.ToTensor(),
            transforms.Normalize((args.mean_image,), (args.std_image,))])
    if not args.augmentation:
        transform_train = transform
        transform_test = transform
    train_loader = torch.utils.data.DataLoader(
            TripletPhotoTour(train=True,
                             load_random_triplets = load_random_triplets,
                             batch_size=args.batch_size,
                             root=args.dataroot,
                             name=args.training_set,
                             download=True,
                             transform=transform_train),
                             batch_size=args.batch_size,
                             shuffle=False, **kwargs)

    test_loaders = [{'name': name,
                     'dataloader': torch.utils.data.DataLoader(
             TripletPhotoTour(train=False,
                     batch_size=args.test_batch_size,
                     root=args.dataroot,
                     name=name,
                     download=True,
                     transform=transform_test),
                        batch_size=args.test_batch_size,
                        shuffle=False, **kwargs)}
                    for name in test_dataset_names]

    return train_loader, test_loaders
print("Done!")
dataset_names = ['liberty', 'notredame', 'yosemite']
train_loader, test_loaders = create_loaders(dataset_names, load_random_triplets = args.load_random_triplets)
print("Done!")

Done!
Done!
Done!
Done!
# Found cached data data/sets/liberty.pt


100%|██████████| 50000/50000 [00:00<00:00, 102261.54it/s]


Generating 50000 triplets
# Found cached data data/sets/notredame.pt
# Found cached data data/sets/yosemite.pt
Done!


---
### Build Network Model
The DesNet is a simple CNN network, which only contains two CNN blocks.

In [11]:
# load network
CNN = 3
model = None
if CNN == 1:
    from descriptor import DesNet_1
    model = DesNet_1()
elif CNN == 2:
    from descriptor import DesNet_2
    model = DesNet_2()
elif CNN == 3:
    from descriptor import DesNet_3
    model = DesNet_3()
elif CNN == 4:
    from descriptor import DesNet_4
    model = DesNet_4()
else:
    print("No such model exists!")
    exit(1)
if model == None:
    print("No model found!")
    exit(1)
if args.cuda:
        model.cuda()
print("Done!")
# define optimizer
def create_optimizer(model, new_lr):
    # setup optimizer
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=new_lr,
                              momentum=0.9, dampening=0.9,
                              weight_decay=args.wd)
    elif args.optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=new_lr,
                               weight_decay=args.wd)
    else:
        raise Exception('Not supported optimizer: {0}'.format(args.optimizer))
    return optimizer
optimizer1 = create_optimizer(model.features, args.lr)
print("Done!")
def train(train_loader, model, optimizer, epoch, logger, load_triplets  = False, name = ""):
    # switch to train mode
    model.train()
    pbar = tqdm(enumerate(train_loader))
    for batch_idx, data in pbar:
        if load_triplets:
            data_a, data_p, data_n = data
        else:
            data_a, data_p = data

        if args.cuda:
            data_a, data_p  = data_a.cuda(), data_p.cuda()
            data_a, data_p = Variable(data_a), Variable(data_p)
            out_a = model(data_a)
            out_p = model(data_p)
        if load_triplets:
            data_n  = data_n.cuda()
            data_n = Variable(data_n)
            out_n = model(data_n)
        
        
        loss = loss_DesNet(out_a, out_p,
                        margin=args.margin,
                        anchor_swap=args.anchorswap,
                        anchor_ave=args.anchorave,
                        batch_reduce = args.batch_reduce,
                        loss_type = args.loss)

        if args.decor:
            loss += CorrelationPenaltyLoss()(out_a)
            
        if args.gor:
            loss += args.alpha*global_orthogonal_regularization(out_a, out_n)
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        adjust_learning_rate(optimizer)
        if batch_idx % args.log_interval == 0:
            pbar.set_description(
                'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data_a), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader),
                    loss.item()))
    out_string = "[" + str(epoch) + ", " + str(loss.item())
    #print(out_string)
    file = open('results-'+str(CNN)+'.txt', 'a')
    file.write(out_string)
    file.close()

    if (args.enable_logging):
#         logger.log_value('loss', loss.data[0]).step()
        logger.log_value('loss', loss.item()).step()

    try:
        os.stat('{}{}'.format(args.model_dir,suffix))
    except:
        os.makedirs('{}{}'.format(args.model_dir,suffix))

    torch.save({'epoch': epoch + 1, 'state_dict': model.state_dict()},
               '{}{}/checkpoint_{}.pth'.format(args.model_dir,suffix,epoch))
    
    
def adjust_learning_rate(optimizer):
    """Updates the learning rate given the learning rate decay.
    The routine has been implemented according to the original Lua SGD optimizer
    """
    for group in optimizer.param_groups:
        if 'step' not in group:
            group['step'] = 0.
        else:
            group['step'] += 1.
        group['lr'] = args.lr * (
        1.0 - float(group['step']) * float(args.batch_size) / (args.n_triplets * float(args.epochs)))
    return
print("Done!")
def test(test_loader, model, epoch, logger, logger_test_name):
    # switch to evaluate mode
    model.eval()

    labels, distances = [], []

    pbar = tqdm(enumerate(test_loader))
    for batch_idx, (data_a, data_p, label) in pbar:
        # data_a.shape= torch.Size([1024, 1, 32, 32]) 
        # data_p.shape =torch.Size([1024, 1, 32, 32]) 
        # label.shape = torch.Size([1024])
        if args.cuda:
            data_a, data_p = data_a.cuda(), data_p.cuda()

        data_a, data_p, label = Variable(data_a, volatile=True), \
                                Variable(data_p, volatile=True), Variable(label)
        out_a = model(data_a)
        out_p = model(data_p)
        dists = torch.sqrt(torch.sum((out_a - out_p) ** 2, 1))  # euclidean distance
        distances.append(dists.data.cpu().numpy().reshape(-1,1))
        ll = label.data.cpu().numpy().reshape(-1, 1)
        labels.append(ll)

        if batch_idx % args.log_interval == 0:
            pbar.set_description(logger_test_name+' Test Epoch: {} [{}/{} ({:.0f}%)]'.format(
                epoch, batch_idx * len(data_a), len(test_loader.dataset),
                       100. * batch_idx / len(test_loader)))

    num_tests = test_loader.dataset.matches.size(0)
    labels = np.vstack(labels).reshape(num_tests)
    distances = np.vstack (distances).reshape(num_tests)

    fpr95 = ErrorRateAt95Recall(labels, 1.0 / (distances + 1e-8))
    print('\33[91mTest set: Accuracy(FPR95): {:.8f}\n\33[0m'.format(fpr95))
    out_string = ", " + str(fpr95)
    if logger_test_name == "notredame":
        out_string += ", "
    else:
        out_string += "],\n"
    #print(out_string)
    file = open('results-'+str(CNN)+'.txt', 'a')
    file.write(out_string)
    file.close()

    if (args.enable_logging):
        logger.log_value(logger_test_name+' fpr95', fpr95)
    return


def ErrorRateAt95Recall(labels, scores):
    distances = 1.0 / (scores + 1e-8)
    recall_point = 0.95
    labels = labels[np.argsort(distances)]
    # Sliding threshold: get first index where recall >= recall_point. 
    # This is the index where the number of elements with label==1 below the threshold reaches a fraction of 
    # 'recall_point' of the total number of elements with label==1. 
    # (np.argmax returns the first occurrence of a '1' in a bool array). 
    threshold_index = np.argmax(np.cumsum(labels) >= recall_point * np.sum(labels)) 

    FP = np.sum(labels[:threshold_index] == 0) # Below threshold (i.e., labelled positive), but should be negative
    TN = np.sum(labels[threshold_index:] == 0) # Above threshold (i.e., labelled negative), and should be negative
    return float(FP) / float(FP + TN)
print("Done!")

Done!
Done!
Done!
Done!


### Training 

In [3]:
args.epochs = 20
start = args.start_epoch
end = start + args.epochs
try:
    logger
except NameError:
    logger, file_logger = None, None
else:
    pass
triplet_flag = args.load_random_triplets
from Losses import loss_DesNet
TEST_ON_W1BS = True
LOG_DIR = args.log_dir
if (args.enable_logging) and (logger == None):
    from Loggers import Logger, FileLogger
    logger = Logger(LOG_DIR)

suffix = '{}_{}_{}'.format(args.experiment_name, args.training_set, args.batch_reduce)
if args.gor:
    suffix = suffix+'_gor_alpha{:1.1f}'.format(args.alpha)
if args.anchorswap:
    suffix = suffix + '_as'
if args.anchorave:
    suffix = suffix + '_av'
if args.fliprot:
        suffix = suffix + '_fliprot'

res_fpr_liberty = torch.zeros(end-start,1)
res_fpr_notredame = torch.zeros(end-start, 1)
res_fpr_yosemite = torch.zeros(end-start, 1)
file = open('results-'+str(CNN)+'.txt', 'w')
file.write("CNN-"+str(CNN)+":\n[Epoch, Loss, Notredame accuracy, Yosemite accuracy]\n")
file.close()

for epoch in range(start, end):
    # iterate over test loaders and test results
    train(train_loader, model, optimizer1, epoch, logger, triplet_flag)
    for test_loader in test_loaders:
        test(test_loader['dataloader'], model, epoch, logger, test_loader['name'])

    #randomize train loader batches
    train_loader, test_loaders2 = create_loaders(dataset_names, load_random_triplets=triplet_flag)
print("Done!")

Train Epoch: 0 [40960/50000 (82%)]	Loss: 0.929034: : 49it [00:17,  2.93it/s]
0it [00:00, ?it/s]/scratch/saeedib/conda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:130: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/scratch/saeedib/conda/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:131: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
notredame Test Epoch: 0 [92160/100000 (92%)]: : 98it [00:19,  5.71it/s]
yosemite Test Epoch: 0 [0/100000 (0%)]: : 1it [00:00,  5.09it/s]

Test set: Accuracy(FPR95): 0.21708000



yosemite Test Epoch: 0 [92160/100000 (92%)]: : 98it [00:20,  5.61it/s]


Test set: Accuracy(FPR95): 0.25834000

# Found cached data data/sets/liberty.pt


100%|██████████| 50000/50000 [00:00<00:00, 100763.14it/s]


...

Test set: Accuracy(FPR95): 0.02994000



yosemite Test Epoch: 19 [92160/100000 (92%)]: : 98it [00:19,  5.20it/s]


Test set: Accuracy(FPR95): 0.07430000

# Found cached data data/sets/liberty.pt


100%|██████████| 50000/50000 [00:00<00:00, 100110.03it/s]


Generating 50000 triplets
# Found cached data data/sets/notredame.pt
# Found cached data data/sets/yosemite.pt
Done!


In [7]:
# load patches
patches = torch.load("../saeedib_keypoints.pt")
print(type(patches))
print(patches.shape)
patches = patches.view(-1, 1, 32, 32).cuda()
print(patches.shape)

print("Done!")

<class 'torch.Tensor'>
torch.Size([5, 200, 1, 32, 32])
torch.Size([1000, 1, 32, 32])
Done!


In [8]:
#Load weights model 1
trained_weight_1 = torch.load("./models/liberty_train/model-1/checkpoint_19.pth")
model.load_state_dict(trained_weight_1['state_dict'])

#Forward pass the patches to the model 1
output1 = model(patches)

#Resize, output1 result
#output.shape = torch.Size([1000, 128])
out1 = output1.view(5, 200, 128).cpu().data

print("Done!")

Done!


In [10]:
#Load weights model 2
trained_weight_2 = torch.load("./models/liberty_train/model-2/checkpoint_19.pth")
model.load_state_dict(trained_weight_2['state_dict'])

#Forward pass the patches to the model 2
output2 = model(patches)

#Resize, output2 result
#output.shape = torch.Size([1000, 128])
out2 = output2.view(5, 200, 128).cpu().data

print("Done!")

Done!


In [12]:
#Load weights model 3
trained_weight_3 = torch.load("./models/liberty_train/model-3/checkpoint_19.pth")
model.load_state_dict(trained_weight_3['state_dict'])

#Forward pass the patches to the model 3
output3 = model(patches)

#Resize, output3 result
#output.shape = torch.Size([1000, 128])
out3 = output3.view(5, 200, 128).cpu().data

print("Done!")

Done!


In [ ]:
#Load weights model 4
trained_weight_4 = torch.load("./models/liberty_train/model-4/checkpoint_19.pth")
model.load_state_dict(trained_weight_4['state_dict'])

#Forward pass the patches to the model 4
output3 = model(patches)

#Resize, output3 result
#output.shape = torch.Size([1000, 128])
out3 = output3.view(5, 200, 128).cpu().data

print("Done!")

In [13]:
#Store themtogether:
all_output = torch.stack((out1, out2, out3))
print(all_output.shape)
output_dir = "../Saeedib_descriptions.pt"
torch.save(all_output, output_dir)
print("Done!")

NameError: name 'Torch' is not defined